# Exploring the dataset

In [ ]:
import pandas as pd
import nltk
import re
import numpy as np
import matplotlib.pyplot as plt
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth =  None
df = pd.read_csv('wiki_movie_plots_deduped.csv')


In [ ]:
queries = [
    "Romantic comedy",
    "Science fiction adventure",
    "Action thriller",
    "Historical drama",
    "Horror mystery",
    "Fantasy epic",
    "Biographical film",
    "Crime investigation",
    "Political thriller",
    "Coming of age",
    "Family friendly",
    "Independent film",
    "War story",
    "Western classic",
    "Documentary",
    "When a flying saucer lands in Washington",
    "One night at Camp Blackfoot, several campers pull a prank",
    "The first Asian Nobel Laureate, Rabindranath Tagore, is still revered",
    "A major international financier is found dead at his Hampshire",
    "Inspector Amar and Inspector Chhaya are after the criminal Hardayal",
    "In a future post-apocalyptic Earth in the year 2293, the",
    "\"Illustrated Press\" society editor Lorelei Kilbourne (Hillary Brooke) is assigned",
    "Sivan (Vinu Mohan) is a medical college student better known",
    "In the prologue, a young Max Skinner, whose parents have",
    "Five men, criminals Ray, Dave, Stevie, Julian (\"Julie\" as a",
    "Prince Edward of Euphrania returns home after meeting the princess",
    "With the outbreak of World War I, a young woman",
    "The Shintai Sobu fitness club is formed by the chief",
    "The city of Kozhikode is victimized by a massive communal",
    "A spy (Essie Lin Chia (fr)) discovers that the Chinese"
]


In [28]:
df.shape

(34886, 8)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34886 non-null  int64 
 1   Title             34886 non-null  object
 2   Origin/Ethnicity  34886 non-null  object
 3   Director          34886 non-null  object
 4   Cast              33464 non-null  object
 5   Genre             34886 non-null  object
 6   Wiki Page         34886 non-null  object
 7   Plot              34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [30]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Smashers,"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Light_of_the_Moon,"The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Presidents,"The film, just over a minute long, is composed of two shots. In the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. At the center of the altar, a viewing portal displays the portraits of three U.S. Presidents—Abraham Lincoln, James A. Garfield, and William McKinley—each victims of assassination.\r\nIn the second shot, which runs just over eight seconds long, an assassin kneels feet of Lady Justice."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_the_Grizzly_King","Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading ""His Photographer"" and ""His Press Agent"" respectively, follow him into the shot; the photographer sets up his camera. ""Teddy"" aims his rifle upward at the tree and fells what appears to be a common house cat, which he then proceeds to stab. ""Teddy"" holds his prize aloft, and the press agent takes notes. The second shot is taken in a slightly different part of the wood, on a path. ""Teddy"" rides the path on his horse towards the camera and out to the left of the shot, followed closely by the press agent and photographer, still dutifully holding their signs."
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1902_film),"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince."


In [31]:
df.isnull().sum()

Release Year           0
Title                  0
Origin/Ethnicity       0
Director               0
Cast                1422
Genre                  0
Wiki Page              0
Plot                   0
dtype: int64

In [32]:
df.isna().sum()

Release Year           0
Title                  0
Origin/Ethnicity       0
Director               0
Cast                1422
Genre                  0
Wiki Page              0
Plot                   0
dtype: int64

In [33]:
print('Number of of unknown values in each column:')
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"{col}: {df[col].str.contains('unknown',case=False,na=True).sum()}")

Number of of unknown values in each column:
Title: 18
Origin/Ethnicity: 0
Director: 1124
Cast: 1423
Genre: 6083
Wiki Page: 18
Plot: 1322


In [34]:
df['Genre'].str.lower().unique()

array(['unknown', 'western', 'comedy', ...,
       'adventure, romance, fantasy film', 'ero',
       'horror romantic comedy'], dtype=object)

In [35]:
len(df['Genre'].str.lower().unique())

2265

In [36]:
df['Genre'].str.lower().value_counts()

Genre
unknown                                                                              6083
drama                                                                                5964
comedy                                                                               4379
horror                                                                               1167
action                                                                               1098
thriller                                                                              966
romance                                                                               923
western                                                                               865
crime                                                                                 568
adventure                                                                             526
musical                                                                               467
crim

In [37]:
len(df['Genre'].str.lower().value_counts())

2265

In [38]:
df[df['Genre'].str.contains('unknown',case=False,na=True)].head(10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Smashers,"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Light_of_the_Moon,"The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Presidents,"The film, just over a minute long, is composed of two shots. In the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. At the center of the altar, a viewing portal displays the portraits of three U.S. Presidents—Abraham Lincoln, James A. Garfield, and William McKinley—each victims of assassination.\r\nIn the second shot, which runs just over eight seconds long, an assassin kneels feet of Lady Justice."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_the_Grizzly_King","Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading ""His Photographer"" and ""His Press Agent"" respectively, follow him into the shot; the photographer sets up his camera. ""Teddy"" aims his rifle upward at the tree and fells what appears to be a common house cat, which he then proceeds to stab. ""Teddy"" holds his prize aloft, and the press agent takes notes. The second shot is taken in a slightly different part of the wood, on a path. ""Teddy"" rides the path on his horse towards the camera and out to the left of the shot, followed closely by the press agent and photographer, still dutifully holding their signs."
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1902_film),"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince."
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderland_(1903_film),"Alice follows a large white rabbit down a ""Rabbit-hole"". She finds a tiny door. When she finds a bottle labeled ""Drink me"", she d

Find the duplicates with the same Title

In [39]:
df[df['Title'].duplicated(keep=False)].shape

(4525, 8)

In [40]:
df[df['Title'].duplicated(keep=False)].head(10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1902_film),"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince."
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderland_(1903_film),"Alice follows a large white rabbit down a ""Rabbit-hole"". She finds a tiny door. When she finds a bottle labeled ""Drink me"", she does, and shrinks, but not enough to pass through the door. She then eats something labeled ""Eat me"" and grows larger. She finds a fan when enables her to shrink enough to get into the ""Garden"" and try to get a ""Dog"" to play with her. She enters the ""White Rabbit's tiny House,"" but suddenly resumes her normal size. In order to get out, she has to use the ""magic fan.""\r\nShe enters a kitchen, in which there is a cook and a woman holding a baby. She persuades the woman to give her the child and takes the infant outside after the cook starts throwing things around. The baby then turns into a pig and squirms out of her grip. ""The Duchess's Cheshire Cat"" appears and disappears a couple of times to Alice and directs her to the Mad Hatter's ""Mad Tea-Party."" After a while, she leaves.\r\nThe Queen invites Alice to join the ""ROYAL PROCESSION"": a parade of marching playing cards and others headed by the White Rabbit. When Alice ""unintentionally offends the Queen"", the latter summons the ""Executioner"". Alice ""boxes the ears"", then flees when all the playing cards come for her. Then she wakes up and realizes it was all a dream."
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before_Christmas_(1905_film),"Scenes are introduced using lines of the poem.[2] Santa Claus, played by Harry Eytinge, is shown feeding real reindeer[4] and finishes his work in the workshop. Meanwhile, the children of a city household hang their stockings and go to bed, but unable to sleep they engage in a pillow fight. Santa Claus leaves his home on a sleigh with his reindeer. He enters the children's house through the chimney, and leaves the presents. The children come down the stairs and enjoy their presents."
12,1906,Kathleen Mavourneen,American,Edwin S. Porter,NaN,short film,https://en.wikipedia.org/wiki/Kathleen_Mavourneen_(1906_film),"Irish villager Kathleen is a tenant of Captain Clearfield, who controls local judges and criminals. Her father owes Clearfield a large debt. Terence O'More saves the village from Clearfield, causing a large celebration.\r\nFilm historian Charles Musser writes of Porter's adaptation, ""O'More not only rescues Kathleen from the villain but, through marriage, renews the family for another generation.""[1]"
15,1907,Laughing Gas,American,Edwin Stanton Porter,"Bertha Regustus, Edward Boulden",comedy,https://en.wikipedia.org/wiki/Laughing_Gas_(film)#1907_Film,"The plot is that of a black woman going to the dentist for a toothache and being given laughing gas. On her way walking home, and in other situations, she can't stop laughing, and everyone she meets ""catches"" the laughter from her, including a vendor and police officers."
19,1908,The Call o

In [41]:
df[df['Title'].str.contains('Jack and the Beanstalk',case=False,na=True)].head(10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1902_film),"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince."
5651,1952,Jack and the Beanstalk,American,Jean Yarbrough,Abbott and Costello,comedy,https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1952_film),"Mr. Dinkle and Jack (Abbott and Costello) look for work at the Cosman Employment Agency. Jack makes advances to Cosman employee Polly (Dorothy Ford), but he is thwarted by the arrival of her boyfriend, a towering police officer (Buddy Baer). Polly assigns Dinkle and Jack to babysit for Eloise Larkin's brother and infant sister, while Eloise (Shaye Cogan) and her fiancé (James Alexander) are out for the evening. The babysitting duties are complicated by the fact that Donald (David Stollery) is something of a prodigy, as well as a self-proclaimed ""problem child"". The dull-witted Jack is soon outclassed by the child, and an attempt to lull the boy to sleep by reading the fairy tale Jack and the Beanstalk (Jack's ""favorite novel"") aloud fails when Jack stumbles over the larger words. Bemused by Jack's incompetence, Donald reads the story instead--a role-reversal made complete when Jack falls asleep as Donald reads. In his slumber, Jack dreams that he is the young Jack of the fairy tale.\r\nIn his dream Jack learns that the Giant (Buddy Baer), who lives in a castle in the sky, has stolen all of the land's wealth and food. The situation obliges the kingdom's princess to marry The Prince (James Alexander) of a neighboring kingdom, whom she has never met.\r\nJack must also make sacrifices, when his mother sends him to sell the last family possession, their beloved cow ""Henry"", to the local butcher, Mr. Dinklepuss. Along the way Jack meets The Prince, disguised as a troubador (who is kidnapped by the Giant soon afterward). The unscrupulous Dinklepuss pays Jack five ""magic"" beans for the cow. Upon returning home, Jack learns that the Giant has also kidnapped The Princess (Shaye Cogan) and Henry.\r\nUndeterred by his mother's disappointment over bringing home only beans, Jack plants them and a gigantic beanstalk grows overnight. He decides to climb the beanstalk to rescue everyone from the Giant's clutches, as well as to retrieve ""Nellie"", the golden-egg laying hen that the Giant previously stole from Jack's family. Upon learning of Nellie's existence, Dinklepuss decides to join Jack on the adventure.\r\nWhen they reach the top of the beanstalk Jack and Dinklepuss are captured by the Giant and imprisoned with the prince and princess. The princess falls for the troubador after he serenades her, only to later learn this is the same prince she was betrothed. After the Giant releases Dinklepuss and Jack from the dungeon in order to toil around his castle, they befriend his housekeeper, Polly, who helps them escape over the castle wall along with the royal prisoners, Nellie and some of the Giant's stolen gems (pilfered by the greedy Dinklepuss). They flee down the beanstalk with the Giant in pursuit, as Polly escapes the castle behind him, astride Henry. During the descent, Dinklepuss loses Nellie (who falls into the arms of Jack's mother) and 

Find the duplicates with the same title and release year

In [42]:
df.duplicated(subset=['Title','Release Year'], keep=False).sum()

np.int64(551)

Remove the duplicates

In [43]:
df.drop_duplicates(subset=['Title', 'Release Year'], keep='first',inplace=True)

check if there are any duplicates title release year

In [44]:
df.duplicated(subset=['Title','Release Year'], keep=False).sum()

np.int64(0)

In [45]:
df.shape

(34608, 8)

Add the release year to the duplicated titles only

In [50]:
duplicates = df[df['Title'].duplicated(keep=False)]

df.loc[duplicates.index, 'Title'] = (
    df.loc[duplicates.index, 'Title'] + ' ' + df.loc[duplicates.index, 'Release Year'].astype(str) 
)
 

check for duplicated titles again

In [52]:
df[df['Title'].duplicated(keep=False)].shape

(0, 8)

In [51]:
df[df['Title'].str.contains('Jack and the Beanstalk',case=False,na=True)].head(10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
4,1902,Jack and the Beanstalk 1902,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1902_film),"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince."
5651,1952,Jack and the Beanstalk 1952,American,Jean Yarbrough,Abbott and Costello,comedy,https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1952_film),"Mr. Dinkle and Jack (Abbott and Costello) look for work at the Cosman Employment Agency. Jack makes advances to Cosman employee Polly (Dorothy Ford), but he is thwarted by the arrival of her boyfriend, a towering police officer (Buddy Baer). Polly assigns Dinkle and Jack to babysit for Eloise Larkin's brother and infant sister, while Eloise (Shaye Cogan) and her fiancé (James Alexander) are out for the evening. The babysitting duties are complicated by the fact that Donald (David Stollery) is something of a prodigy, as well as a self-proclaimed ""problem child"". The dull-witted Jack is soon outclassed by the child, and an attempt to lull the boy to sleep by reading the fairy tale Jack and the Beanstalk (Jack's ""favorite novel"") aloud fails when Jack stumbles over the larger words. Bemused by Jack's incompetence, Donald reads the story instead--a role-reversal made complete when Jack falls asleep as Donald reads. In his slumber, Jack dreams that he is the young Jack of the fairy tale.\r\nIn his dream Jack learns that the Giant (Buddy Baer), who lives in a castle in the sky, has stolen all of the land's wealth and food. The situation obliges the kingdom's princess to marry The Prince (James Alexander) of a neighboring kingdom, whom she has never met.\r\nJack must also make sacrifices, when his mother sends him to sell the last family possession, their beloved cow ""Henry"", to the local butcher, Mr. Dinklepuss. Along the way Jack meets The Prince, disguised as a troubador (who is kidnapped by the Giant soon afterward). The unscrupulous Dinklepuss pays Jack five ""magic"" beans for the cow. Upon returning home, Jack learns that the Giant has also kidnapped The Princess (Shaye Cogan) and Henry.\r\nUndeterred by his mother's disappointment over bringing home only beans, Jack plants them and a gigantic beanstalk grows overnight. He decides to climb the beanstalk to rescue everyone from the Giant's clutches, as well as to retrieve ""Nellie"", the golden-egg laying hen that the Giant previously stole from Jack's family. Upon learning of Nellie's existence, Dinklepuss decides to join Jack on the adventure.\r\nWhen they reach the top of the beanstalk Jack and Dinklepuss are captured by the Giant and imprisoned with the prince and princess. The princess falls for the troubador after he serenades her, only to later learn this is the same prince she was betrothed. After the Giant releases Dinklepuss and Jack from the dungeon in order to toil around his castle, they befriend his housekeeper, Polly, who helps them escape over the castle wall along with the royal prisoners, Nellie and some of the Giant's stolen gems (pilfered by the greedy Dinklepuss). They flee down the beanstalk with the Giant in pursuit, as Polly escapes the castle behind him, astride Henry. During the descent, Dinklepuss loses Nellie (who falls into the arms of Jack's mo

# Preprocessing

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text (text ,remove_stopwords=True, lowercase=True,stemming=False, lemmatization=False):

    if lowercase:
        text = text.lower()
    
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    
    tokens = word_tokenize(text)
    
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

    
    return ' '.join(tokens)



# Text Representation

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


## BM25

## Word2Vec

## Bert

# Indexing

# Ranking Results